In [1]:
import random
import math

nucleotides = ["A" , "C" , "G" ,"T" ]
sequences = []

##Functions of PSSM

In [2]:
def PSSM(seq_num, seq_len, sequences):
    # Create a frequency table where nucleotides_freq[j] represents the counts of A, C, G, and T at the j-th position
    # across all sequences.
    nucleotides_freq = [[0.0 for _ in range(len(nucleotides))] for _ in range(seq_len)]

    # Iterate over each sequence
    for sequence in sequences:
        # Loop through each position in the sequence
        for j in range(seq_len):
            if sequence[j] in nucleotides:
                # Update the count for the current nucleotide at the current position
                nucleotides_freq[j][nucleotides.index(sequence[j])] += 1
            else:
                print("Invalid input. Please enter sequences containing only 'A', 'G', 'C', 'T' nucleotides.")
                return None  # Exit function if invalid nucleotide found

    # Calculate overall frequencies for normalization
    total_positions = seq_num * seq_len
    overall_freqs = [sum(col) / total_positions for col in zip(*nucleotides_freq)]

    # Normalize nucleotide counts by overall frequencies (PSSM calculation)
    pssm_matrix = [[nucleotides_freq[j][i] / overall_freqs[i] for i in range(len(nucleotides))] for j in range(seq_len)]

    # Calculate the log probabilities (base 2) from normalized frequencies
    log_prob_freq = [[round(math.log2(max(f, 1e-10)), 1) for f in row] for row in pssm_matrix]

    return log_prob_freq

#Calculating the score for a new sequence based on PSSM

def calc_score(new_seq, pssm_matrix):
    score = 0.0
    # Loop through each position and nucleotide in the sequence
    for i, nucleotide in enumerate(new_seq):
        if nucleotide in "ACGT":
            # Find the index of the nucleotide in the nucleotides list
            index = nucleotides.index(nucleotide)
            # Access the log probability for the nucleotide at the corresponding position
            log_prob = pssm_matrix[i][index]
            # Add log probabilities
            score += log_prob
        else:
            print("Invalid input. Please enter sequences containing only 'A', 'G', 'C', 'T' nucleotides.")
            return None  # Exit function if invalid nucleotide found
    return score


###Dispaly the menu

In [5]:
print("\nPosition-Specific Scoring Matrix ( PSSM ) Analyzer.\n\nChoose input method : ")
print("1.Read the DNA sequences input from the file ( PSSMData.txt ).\n2.Generate random sequences.")
choice = int(input("\nEnter your choice : "))

# 2.Getting the DNA seq from one way based on user choice
if choice == 1:
    filename = "PSSMData.txt"
    with open(filename, "r") as file:
        print(f"\nReading input from the file ...")
        # To read the a single line ( which is the 1st one) in the filey
        # Then, remove the whitespaces
        # Finally, splitting the content in a list using whitespace delimiter
        first_line = file.readline().strip().split()
        seq_num = int(first_line[0])
        seq_len = int(first_line[1])
        # Read DNA sequences from the file, capitalize, and strip whitespace
        sequences = [file.readline().upper().strip() for _ in range(seq_num)]
        print(f"\nFound {seq_num} sequences of length {seq_len}.")

elif choice == 2:
    seq_num = int(input("Enter the number of desired sequences ( t ) : "))
    seq_len = int(input("Enter the length of each DNA sequence ( n ) : "))
    print(f"Generating {seq_num} random sequences of with length for each {seq_len}...")
    sequences = ["".join([random.choice(nucleotides) for _ in range(seq_len)]) for _ in range(seq_num)]

else:
    print("Invalid input. Please, enter 1 or 2.")

#3.Printing the DNA sequences
print("\nMultiple Aligned DNA sequences : ")
for i in range(len(sequences)):
    print(f"Sequence {i + 1} : {sequences[i]}")

#4. Applying the PSSM Algorithm
print("\nApplying PSSM...")

pssm_matrix = PSSM(seq_num, seq_len, sequences)
if pssm_matrix is not None:
    print("\n{:>4} {:>10} {:>10} {:>10} {:>10}".format(" ", "A", "T", "C", "G"))
    for i in range(seq_num):
        print(f"{i + 1:>4} {pssm_matrix[i][0]:>10.3f} {pssm_matrix[i][3]:>10.3f} {pssm_matrix[i][1]:>10.3f} {pssm_matrix[i][2]:>10.3f}")


#5.Entering the new sequence
new_seq = input(f"\nEnter a new sequence of length {seq_len} : ").upper().strip()
if len(new_seq) != seq_len:
    print(f"Invalid sequence. Please enter a sequence of length {seq_len}.")
else:
    # 6.Calculating the probability  of the new entered sequence joining the rest of the multiple aligned DNA sequences based on the PSSM
    new_seq_prob = calc_score(new_seq, pssm_matrix)
    if new_seq_prob is not None:
        print(f"The log probability of the new entered sequence joining the alignment: {new_seq_prob:.2f}")


Position-Specific Scoring Matrix ( PSSM ) Analyzer.

Choose input method : 
1.Read the DNA sequences input from the file ( PSSMData.txt ).
2.Generate random sequences.

Enter your choice : 1

Reading input from the file ...

Found 5 sequences of length 8.

Multiple Aligned DNA sequences : 
Sequence 1 : AGGTACTT
Sequence 2 : CCATACGT
Sequence 3 : ACGTTAGT
Sequence 4 : ACGTCCAT
Sequence 5 : CCGTACGG

Applying PSSM...

              A          T          C          G
   1      3.700    -33.200      2.900    -33.200
   2    -33.200    -33.200      3.900      2.200
   3      2.200    -33.200    -33.200      4.200
   4    -33.200      4.200    -33.200    -33.200
   5      3.700      1.900      1.900    -33.200

Enter a new sequence of length 8 : CCGTACGG
The log probability of the new entered sequence joining the alignment: 28.70
